In [1]:
from evaluate_task import evaluate_multiple_choice_task 
from evaluate_task import evaluate_free_response_task
GPT3_5 = "gpt-3.5-turbo-1106"
GPT4 = "gpt-4-0125-preview"

#Task 1 - multiple choice
task_anachronisms = 'data/task_anachronisms.json'
sys_anachronism = "Does the following sentence contain non-contemporaneous (anachronistic) elements?."

#Task 2 - multiple choice
task_logical_sequence = 'data/task_logical_sequence.json'
sys_logical_sequence = "Identify the correct chronological or sequential order of items in a list."

#Task 3 - multiple choice
task_color_HCL = 'data/task_color_HCL.json'
task_color_RGB = 'data/task_color_RGB.json'
task_color_HEX = 'data/task_color_HEX.json'
task_color_HSL = 'data/task_color_HSL.json'

#Task 4 - free choice
task_matrixshapes = 'data/task_matrixshapes.json'
sys_matrixshapes = "Keep track of matrix shapes through various transformations."

#Task 5 - free choice
task_operators = 'data/task_operators.json'
sys_operators = "Given the definition of the op operator, compute the result."

In [ ]:
#print(f"Accuracy#1: {evaluate_multiple_choice_task(task_anachronisms, sys_anachronism, GPT3_5)}")
#print(f"Accuracy#2: {evaluate_multiple_choice_task(task_logical_sequence, sys_logical_sequence, GPT3_5)}")
print(f"Accuracy#3.1: {evaluate_multiple_choice_task(task_color_HCL, "", GPT3_5)}")
print(f"Accuracy#3.2: {evaluate_multiple_choice_task(task_color_RGB, "", GPT3_5)}")
print(f"Accuracy#3.3: {evaluate_multiple_choice_task(task_color_HEX, "", GPT3_5)}")
print(f"Accuracy#3.4: {evaluate_multiple_choice_task(task_color_HSL, "", GPT3_5)}")

In [2]:
print(f"Accuracy#4: {evaluate_free_response_task(task_matrixshapes, sys_matrixshapes, GPT3_5)}")

Accuracy#4: 1.0


In [ ]:
import wandb
# start a new wandb run to track this script
wandb.init(project="LLMs-Still-Can-t",config={
    "model": GPT3_5,
    "dataset": task_anachronisms,
    "shot": 0})

# evaluate the task
acc = evaluate_multiple_choice_task(task_anachronisms, sys_anachronism, GPT3_5)


# log the final accuracy back to wandb, and end the run
wandb.log({"acc": acc, "results": }) 
wandb.finish()